In [8]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold,StratifiedKFold
import re

In [2]:
import tensorflow as tf
import keras

Using Theano backend.


In [3]:
data_path = '/home/raku/kaggleData/2sigma/lr4/'
store = '/home/raku/kaggleData/2sigma/nn'

train_df=pd.read_json(data_path+'lr4-n-train.json')
test_df=pd.read_json(data_path+'lr4-n-test.json')



In [4]:
pickl_file = '/home/raku/kaggleData/2sigma/loglr/'+'loglrfeatures.pickle'
fileObject = open(pickl_file,'r') 
features=pickle.load(fileObject)   
fileObject.close()
print len(features)

223


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras import optimizers
from keras.callbacks import EarlyStopping

In [6]:
def nn_model(features,num_classes=3,lr=0.1):
    model = Sequential()
    model.add(Dense(64,  
                    activation='softplus',
                    input_shape = (len(features),),
                                  kernel_initializer='he_normal',
                                  kernel_regularizer=l2(0.000025)
                                  ))
    model.add(Dropout(0.2))
    
    model.add(Dense(16,
                    activation='softplus', 
                    kernel_initializer='he_normal',
                    kernel_regularizer=l2(0.000025)
                    ))
    model.add(Dropout(0.1))

    model.add(Dense(units=num_classes, 
                    activation='softmax', 
                    kernel_initializer='he_normal',
                    ))
    opt = optimizers.Adadelta(lr=1)
    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer=opt,
                  metrics=['accuracy']
                  )
    return(model)

In [9]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 2333)



In [11]:


for batch_size in [64]:

    i=0
    cv_scores=[]
    cv_result=[]
    for dev_index, val_index in KF:
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
    
        early_stopping = EarlyStopping(monitor='val_loss', patience=20)

        seed = 0
        np.random.seed(seed)
        model = nn_model(features,lr=0.1)
        history=model.fit(dev_X, dev_y, epochs = 1000, batch_size=batch_size, verbose = 2, 
          validation_data=[val_X, val_y], callbacks=[early_stopping])

        preds =  model.predict_proba(val_X)
        """
        #save the pickles for futures use
        pickl_file = store+'nn-5fold-out-'+str(i)+'.pickle'
        fileObject = open(pickl_file,'wb') 
        pickle.dump(preds,fileObject)   
        fileObject.close()    
        """
        lls=log_loss(val_y, preds)
        cv_scores.append(lls)
        cv_result.append(history)
        i+=1
        print 'the cv_score for the '+str(i)+' turn is:'
        print(lls)
    
    print np.mean(cv_scores)


Train on 39481 samples, validate on 9871 samples
Epoch 1/1000
8s - loss: 0.7119 - acc: 0.7015 - val_loss: 0.6261 - val_acc: 0.7236
Epoch 2/1000
9s - loss: 0.6252 - acc: 0.7243 - val_loss: 0.6121 - val_acc: 0.7269
Epoch 3/1000
9s - loss: 0.6096 - acc: 0.7309 - val_loss: 0.6084 - val_acc: 0.7297
Epoch 4/1000
8s - loss: 0.6038 - acc: 0.7346 - val_loss: 0.6044 - val_acc: 0.7292
Epoch 5/1000
8s - loss: 0.5984 - acc: 0.7358 - val_loss: 0.6031 - val_acc: 0.7283
Epoch 6/1000
9s - loss: 0.5959 - acc: 0.7377 - val_loss: 0.6019 - val_acc: 0.7294
Epoch 7/1000
7s - loss: 0.5930 - acc: 0.7381 - val_loss: 0.6009 - val_acc: 0.7295
Epoch 8/1000
10s - loss: 0.5889 - acc: 0.7400 - val_loss: 0.5997 - val_acc: 0.7327
Epoch 9/1000
9s - loss: 0.5876 - acc: 0.7397 - val_loss: 0.5991 - val_acc: 0.7344
Epoch 10/1000
9s - loss: 0.5874 - acc: 0.7411 - val_loss: 0.5971 - val_acc: 0.7343
Epoch 11/1000
9s - loss: 0.5860 - acc: 0.7414 - val_loss: 0.5958 - val_acc: 0.7345
Epoch 12/1000
7s - loss: 0.5827 - acc: 0.7415 

In [12]:
cv_result

In [20]:
cv_result[0].history['val_loss']

[0.62606498579427761,
 0.61213838925490616,
 0.60841578445636491,
 0.60436816580715103,
 0.60310107293842463,
 0.60187766285921329,
 0.60087456179418752,
 0.59972067699160891,
 0.59913021746334871,
 0.59711539933123781,
 0.59582158282177267,
 0.59593743994030446,
 0.59455101442438485,
 0.59542512690184046,
 0.59469518580222702,
 0.5923505682324991,
 0.59261432500514033,
 0.59269209234031694,
 0.59263453171639902,
 0.58997414772929113,
 0.59098968372653171,
 0.59063844262448595,
 0.59226828722373626,
 0.58984572048571782,
 0.59089995122537975,
 0.58936574094117489,
 0.58871431120525752,
 0.58916067796907534,
 0.58955185714795488,
 0.58879555207643697,
 0.58815046192338649,
 0.58825219645469751,
 0.58760493066871,
 0.58978645340938141,
 0.58824349365726136,
 0.58774308901211858,
 0.58860097692532332,
 0.58907870870268875,
 0.58758846954569466,
 0.58762531247427374,
 0.58785817132461349,
 0.58869448884764575,
 0.58665946105755684,
 0.58728366602325788,
 0.58638138355497238,
 0.58817404180

In [24]:
test={}
for i in range(len(cv_result)):
    test[i]=cv_result[i].history['val_loss'][:59]


In [25]:
val_history=pd.DataFrame(test)

In [29]:
np.mean(val_history,axis=1)

0     0.614080
1     0.600461
2     0.595790
3     0.592497
4     0.590640
5     0.589203
6     0.588110
7     0.587252
8     0.586053
9     0.585096
10    0.584148
11    0.583692
12    0.582710
13    0.582650
14    0.581915
15    0.581202
16    0.580591
17    0.580409
18    0.579975
19    0.578951
20    0.578642
21    0.578572
22    0.578416
23    0.577913
24    0.577859
25    0.577238
26    0.577030
27    0.576846
28    0.576900
29    0.577103
30    0.575953
31    0.576324
32    0.575653
33    0.576207
34    0.575818
35    0.575610
36    0.575744
37    0.575663
38    0.574865
39    0.575395
40    0.575109
41    0.575079
42    0.574784
43    0.574955
44    0.574745
45    0.574985
46    0.575029
47    0.574945
48    0.575239
49    0.575043
50    0.574956
51    0.575428
52    0.576071
53    0.575606
54    0.575508
55    0.575538
56    0.575587
57    0.574966
58    0.575614
dtype: float64

In [30]:
for batch_size in [64]:

    i=0
    cv_scores=[]
    cv_result=[]
    for dev_index, val_index in KF:
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
    
        early_stopping = EarlyStopping(monitor='val_loss', patience=20)

        seed = 0
        np.random.seed(seed)
        model = nn_model(features,lr=0.1)
        history=model.fit(dev_X, dev_y, epochs = 40, batch_size=batch_size, verbose = 2 
          #,validation_data=[val_X, val_y], callbacks=[early_stopping]
                         )
        preds =  model.predict_proba(val_X)
    
            #save the pickles for futures use
        pickl_file = store+'nn-5fold-out-'+str(i)+'.pickle'
        fileObject = open(pickl_file,'wb') 
        pickle.dump(preds,fileObject)   
        fileObject.close()    
        
        lls=log_loss(val_y, preds)
        cv_scores.append(lls)
        cv_result.append(history)
        i+=1
        print 'the cv_score for the '+str(i)+' turn is:'
        print(lls)
    
    print np.mean(cv_scores)

Epoch 1/40
14s - loss: 0.7119 - acc: 0.7015
Epoch 2/40
15s - loss: 0.6268 - acc: 0.7254
Epoch 3/40
17s - loss: 0.6083 - acc: 0.7319
Epoch 4/40
13s - loss: 0.6014 - acc: 0.7351
Epoch 5/40
13s - loss: 0.5984 - acc: 0.7362
Epoch 6/40
17s - loss: 0.5960 - acc: 0.7379
Epoch 7/40
14s - loss: 0.5936 - acc: 0.7382
Epoch 8/40
12s - loss: 0.5901 - acc: 0.7391
Epoch 9/40
13s - loss: 0.5892 - acc: 0.7394
Epoch 10/40
15s - loss: 0.5856 - acc: 0.7404
Epoch 11/40
16s - loss: 0.5854 - acc: 0.7404
Epoch 12/40
12s - loss: 0.5834 - acc: 0.7428
Epoch 13/40
16s - loss: 0.5831 - acc: 0.7433
Epoch 14/40
15s - loss: 0.5810 - acc: 0.7417
Epoch 15/40
14s - loss: 0.5808 - acc: 0.7424
Epoch 16/40
9s - loss: 0.5798 - acc: 0.7430
Epoch 17/40
14s - loss: 0.5764 - acc: 0.7437
Epoch 18/40
17s - loss: 0.5757 - acc: 0.7431
Epoch 19/40
16s - loss: 0.5765 - acc: 0.7447
Epoch 20/40
15s - loss: 0.5760 - acc: 0.7445
Epoch 21/40
12s - loss: 0.5745 - acc: 0.7451
Epoch 22/40
17s - loss: 0.5747 - acc: 0.7456
Epoch 23/40
16s - lo

In [31]:
"""trainX testX for et and rf """
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

seed = 0
np.random.seed(seed)
model = nn_model(features,lr=0.1)
history=model.fit(train_X, train_y, epochs = 40, batch_size=64, verbose = 2) 
  #,validation_data=[val_X, val_y])#, callbacks=[early_stopping])

preds =  model.predict_proba(test_X)

out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'nn-bulk-out.json')

Epoch 1/40
8s - loss: 0.6997 - acc: 0.7037
Epoch 2/40
8s - loss: 0.6215 - acc: 0.7276
Epoch 3/40
7s - loss: 0.6106 - acc: 0.7302
Epoch 4/40
8s - loss: 0.6039 - acc: 0.7332
Epoch 5/40
7s - loss: 0.5993 - acc: 0.7337
Epoch 6/40
9s - loss: 0.5948 - acc: 0.7370
Epoch 7/40
8s - loss: 0.5931 - acc: 0.7375
Epoch 8/40
8s - loss: 0.5910 - acc: 0.7387
Epoch 9/40
9s - loss: 0.5896 - acc: 0.7372
Epoch 10/40
9s - loss: 0.5878 - acc: 0.7412
Epoch 11/40
9s - loss: 0.5865 - acc: 0.7403
Epoch 12/40
8s - loss: 0.5853 - acc: 0.7403
Epoch 13/40
7s - loss: 0.5840 - acc: 0.7419
Epoch 14/40
8s - loss: 0.5835 - acc: 0.7410
Epoch 15/40
7s - loss: 0.5812 - acc: 0.7416
Epoch 16/40
10s - loss: 0.5816 - acc: 0.7425
Epoch 17/40
8s - loss: 0.5801 - acc: 0.7420
Epoch 18/40
8s - loss: 0.5801 - acc: 0.7436
Epoch 19/40
8s - loss: 0.5788 - acc: 0.7415
Epoch 20/40
9s - loss: 0.5781 - acc: 0.7448
Epoch 21/40
8s - loss: 0.5757 - acc: 0.7459
Epoch 22/40
8s - loss: 0.5769 - acc: 0.7430
Epoch 23/40
8s - loss: 0.5744 - acc: 0.7

In [33]:
out_df.to_json(store+'nn-bulk-out.json')